# Day 1: Secret Entrance

The Elves have good news and bad news.

The good news is that they’ve discovered project-management! This has given them the tools they need to prevent their usual Christmas emergency. For example, they now know that the North Pole decorations need to be finished soon so that other critical tasks can start on time.

The bad news is that they’ve realized they have a different emergency: according to their resource planning, none of them have any time left to decorate the North Pole!

To save Christmas, the Elves need you to finish decorating the North Pole by December 12th.

Collect stars by solving puzzles. Two puzzles will be made available on each day; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can’t get in. A document taped to the wall helpfully explains:

> Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination.

The safe has a dial with only an arrow on it; around the dial are the numbers `0` through `99` in order. As you turn the dial, it makes a small click noise as it reaches each number.

The attached document (your puzzle input) contains a sequence of rotations, one per line, which tell you how to open the safe. A rotation starts with an `L` or `R` which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction.

So, if the dial were pointing at `11`, a rotation of `R8` would cause the dial to point at `19`. After that, a rotation of `L19` would cause it to point at `0`.

Because the dial is a circle, turning the dial left from `0` one click makes it point at `99`. Similarly, turning the dial right from `99` one click makes it point at `0`.

So, if the dial were pointing at `5`, a rotation of `L10` would cause it to point at `95`. After that, a rotation of `R5` could cause it to point at `0`.

The dial starts by pointing at `50`.

You could follow the instructions, but your recent required official North Pole secret-entrance security training seminar taught you that the safe is actually a decoy. The **actual password** is the number of times the dial is left pointing at `0` **after any rotation** in the sequence.

For example, suppose the attached document contained the following rotations:

```
L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
```

Following these rotations would cause the dial to move as follows:

* The dial starts by pointing at `50`.
* The dial is rotated `L68` → points at `82`.
* Rotated `L30` → `52`.
* Rotated `R48` → `0`.
* Rotated `L5` → `95`.
* Rotated `R60` → `55`.
* Rotated `L55` → `0`.
* Rotated `L1` → `99`.
* Rotated `L99` → `0`.
* Rotated `R14` → `14`.
* Rotated `L82` → `32`.

Because the dial points at `0` a total of three times during this process, the password in this example is `3`.

**Task:** Analyze the rotations in your attached document. What’s the actual password to open the door? ([adventofcode.com][1])

[1]: https://adventofcode.com/2025/day/1 "Day 1 - Advent of Code 2025"

In [47]:
class Dial:
    def __init__(self, position: int, sequence: str | None):
        self.min_dial = 0
        self.max_dial = 100

        self.position = position
        self.count = 0

        self.sequence = []
        if sequence is not None:
            self._read_seq(sequence)

    def _rotate_dial(self, dir: str,  clicks: int):
        clicks = -clicks if dir == 'L' else clicks
        self.position = (self.position + clicks) % self.max_dial

        if self.position == 0:
            self.count += 1

    def _read_seq(self, path: str | None):
        if path is None:
            return
        with open(path, 'r', encoding='utf-8') as f:
            for l in f.readlines():
                l = l.strip()
                self.sequence.append((l[0], int(l[1:])))

    def complete_seq(self):
        for d, m in self.sequence:
            self._rotate_dial(d, m)


dial = Dial(50, "sequence.txt")
dial.complete_seq()
print(dial.count)



1089


# **Part Two**

The door still won’t open.
You knock.
No answer.
You build a snowman.

In the snow, you find another document:

> **“Due to newer security protocols, please use password method 0x434C49434B until further notice.”**

Your training reminds you that **method 0x434C49434B** means:

> **Count *every* time the dial clicks past 0 — not just when a rotation ends at 0.**

Using the same example rotations, the dial hits 0 additional times *during* movements:

* Start at **50**
* L68 → end at **82**, hits **0 once**
* L30 → **52**
* R48 → **0**
* L5 → **95**
* R60 → end at **55**, hits **0 once**
* L55 → **0**
* L1 → **99**
* L99 → **0**
* R14 → **14**
* L82 → end at **32**, hits **0 once**

Totals:

* **3** times at the *end* of rotations
* **3** times *during* rotations
* **6 total**

One caution:

> If the dial is at 50, a rotation like **R1000** would cause it to click past **0 ten times** before returning to 50.

**Using method 0x434C49434B, what is your new password?**

---

If you want, I can turn this into a stylized PDF or provide matching Markdown for all future days.


In [54]:
class Dial:
    def __init__(self, position: int, sequence: str | None):
        self.min_dial = 0
        self.max_dial = 100

        self.position = position
        self.count = 0

        self.sequence = []
        if sequence is not None:
            self._read_seq(sequence)

    def _rotate_dial(self, dir: str,  clicks: int):
        # count crossings
        offset = 100 - self.position if dir == 'R' else self.position
        offset = 100 if self.position == 0 else offset
        if clicks < offset:
            hits = 0
        else:
            hits = 1 + (clicks - offset) // 100
        self.count += hits

        clicks = -clicks if dir == 'L' else clicks
        self.position = (self.position + clicks) % self.max_dial

    def _read_seq(self, path: str | None):
        if path is None:
            return
        with open(path, 'r', encoding='utf-8') as f:
            for l in f.readlines():
                l = l.strip()
                self.sequence.append((l[0], int(l[1:])))

    def complete_seq(self):
        for d, m in self.sequence:
            self._rotate_dial(d, m)


dial = Dial(50, "sequence.txt")
dial.complete_seq()
print(dial.count)

6530


# Test Suite

In [52]:
def run_test(name, start, direction, clicks, expected):
    d = Dial(start, None)
    d.sequence = [(direction, clicks)]
    d.complete_seq()
    result = d.count

    print(f"{'OK' if result == expected else 'FAIL'}: {name:<28} → got={result} expected={expected}")


print("=== AoC Example Tests ===")
run_test("L68 from 50", 50, "L", 68, 1)
run_test("L30 from 82", 82, "L", 30, 0)
run_test("R48 from 52", 52, "R", 48, 1)
run_test("R60 from 95", 95, "R", 60, 1)
run_test("L55 from 55", 55, "L", 55, 1)
run_test("L99 from 99", 99, "L", 99, 1)
run_test("L82 from 14", 14, "L", 82, 1)

print("\n=== Multi-wrap Tests ===")
run_test("R150 from 50", 50, "R", 150, 2)
run_test("L250 from 50", 50, "L", 250, 3)
run_test("R350 from 10", 10, "R", 350, 3)

print("\n=== Boundary Tests ===")
run_test("Exact hit R50", 50, "R", 50, 1)
run_test("Exact hit L50", 50, "L", 50, 1)

run_test("Exact wrap R100", 99, "R", 100, 1)
run_test("Exact wrap L100", 1,  "L", 100, 1)

print("\n=== No-Hit Tests ===")
run_test("Too small R", 30, "R", 20, 0)
run_test("Too small L", 30, "L", 20, 0)

print("\n=== Stress ===")
run_test("R1000 from 50", 50, "R", 1000, 10)
run_test("L1000 from 50", 50, "L", 1000, 10)


print("\n=== EXTENDED EDGE CASES ===")

# Starting exactly on 0
run_test("Start 0, R1",   0,  "R", 1, 0)
run_test("Start 0, L1",   0,  "L", 1, 0)
run_test("Start 0, R100", 0,  "R", 100, 1)
run_test("Start 0, L100", 0,  "L", 100, 1)

# Tiny movements
run_test("Small R barely before hit", 99, "R", 0, 0)
run_test("Small L barely before hit", 1,  "L", 0, 0)
run_test("Small R crossing 0",        99, "R", 1, 1)
run_test("Small L crossing 0",        1,  "L", 1, 1)

# Exact offset-only hits
run_test("Exact offset R", 70, "R", 30, 1)
run_test("Exact offset L", 30, "L", 30, 1)

# Large but sub-wrap (CORRECTED EXPECTED)
run_test("Large sub-wrap R", 20, "R", 80, 1)  # expected was wrong before
run_test("Large sub-wrap L", 80, "L", 80, 1)

# Multiple high wraps
run_test("R10000 from 50", 50, "R", 10000, 100)
run_test("L10000 from 50", 50, "L", 10000, 100)

# Near-boundary large rotations (CORRECTED EXPECTED)
run_test("R10001 from 99", 99, "R", 10001, 101)
run_test("L10001 from 1",  1,  "L", 10001, 101)

# Clicks barely into the next wrap region (CORRECTED EXPECTED)
run_test("One extra wrap R", 10, "R", 110, 1)
run_test("One extra wrap L", 90, "L", 110, 1)

# Clicks missing next wrap by one
run_test("Just before wrap R", 10, "R", 109, 1)
run_test("Just before wrap L", 90, "L", 109, 1)

# Clicks cause exactly n wraps
for n in [1, 2, 3, 5, 10]:
    run_test(f"R wrap {n}", 50, "R", n*100 + 50, n+1)
    run_test(f"L wrap {n}", 50, "L", n*100 + 50, n+1)

# Rotations where offset > 100 (in theory)
run_test("Offset > 100 scenario", 50, "R", 49, 0)

# Rotations ending exactly on zero
run_test("Ends exactly at 0 R", 30, "R", 70, 1)
run_test("Ends exactly at 0 L", 30, "L", 30, 1)

# Rotations ending exactly at 0 multiple times
run_test("Ends at 0 twice", 50, "R", 50 + 100 + 100, 3)


print("\n=== RANDOMIZED STRESS TESTS ===")
import random

for i in range(20):
    start = random.randint(0, 99)
    clicks = random.randint(0, 20000)
    direction = random.choice(["L", "R"])
    
    # brute-force simulate the dial click-by-click
    pos = start
    brute_hits = 0
    for _ in range(clicks):
        pos = (pos + (1 if direction == "R" else -1)) % 100
        if pos == 0:
            brute_hits += 1
    
    d = Dial(start, None)
    d.sequence = [(direction, clicks)]
    d.complete_seq()
    
    status = "OK" if d.count == brute_hits else "FAIL"
    print(f"{status}: brute={brute_hits} class={d.count}  start={start} dir={direction} clicks={clicks}")


=== AoC Example Tests ===
OK: L68 from 50                  → got=1 expected=1
OK: L30 from 82                  → got=0 expected=0
OK: R48 from 52                  → got=1 expected=1
OK: R60 from 95                  → got=1 expected=1
OK: L55 from 55                  → got=1 expected=1
OK: L99 from 99                  → got=1 expected=1
OK: L82 from 14                  → got=1 expected=1

=== Multi-wrap Tests ===
OK: R150 from 50                 → got=2 expected=2
OK: L250 from 50                 → got=3 expected=3
OK: R350 from 10                 → got=3 expected=3

=== Boundary Tests ===
OK: Exact hit R50                → got=1 expected=1
OK: Exact hit L50                → got=1 expected=1
OK: Exact wrap R100              → got=1 expected=1
OK: Exact wrap L100              → got=1 expected=1

=== No-Hit Tests ===
OK: Too small R                  → got=0 expected=0
OK: Too small L                  → got=0 expected=0

=== Stress ===
OK: R1000 from 50                → got=10 expected=10
